In [ ]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import itertools
import warnings

cf10

In [ ]:
df_path_cf10 = 'cf10-rn/ww-df'

In [ ]:
def delta_merge_layers(folder_path,suffix):
    # Initialize dictionaries to store DataFrames for ft and non-ft files
    dataframes_ft = {}
    dataframes_non_ft = {}

    # List all CSV files in the folder
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Loop through each file and categorize it as 'ft' or 'non-ft'
    for file in files:
        # Check if file matches the ft pattern
        if f'_{suffix}_ft.csv' in file:
            base_name = file.replace(f'_{suffix}_ft.csv', '')
            file_path = os.path.join(folder_path, file)
            # Read the entire DataFrame from the ft file
            df = pd.read_csv(file_path)
            if not df.empty:
                dataframes_ft[base_name] = df
        # Check if file matches the non-ft pattern
        elif f'_{suffix}.csv' in file:
            base_name = file.replace(f'_{suffix}.csv', '')
            file_path = os.path.join(folder_path, file)
            # Read the entire DataFrame from the non-ft file
            df = pd.read_csv(file_path)
            if not df.empty:
                dataframes_non_ft[base_name] = df

    # Initialize a list to store the differences DataFrames
    list_differences = []

    # Compute the differences for each matching pair
    for base_name in dataframes_ft:
        if base_name in dataframes_non_ft:
            df_ft = dataframes_ft[base_name]
            df_non_ft = dataframes_non_ft[base_name]

            # Align DataFrames on both axes (rows and columns)
            df_ft_aligned, df_non_ft_aligned = df_ft.align(df_non_ft, join='outer', axis=None, fill_value=0)

            # Convert boolean columns to integers
            for df in [df_ft_aligned, df_non_ft_aligned]:
                bool_cols = df.select_dtypes(include=['bool']).columns
                df[bool_cols] = df[bool_cols].astype(int)

            # Convert all columns to numeric, coercing errors to NaN
            df_ft_numeric = df_ft_aligned.apply(pd.to_numeric, errors='coerce')
            df_non_ft_numeric = df_non_ft_aligned.apply(pd.to_numeric, errors='coerce')

            # Fill NaN values with 0 (optional, depending on how you want to handle missing values)
            df_ft_numeric = df_ft_numeric.fillna(0)
            df_non_ft_numeric = df_non_ft_numeric.fillna(0)

            # Compute the difference between the ft and non-ft DataFrames
            df_diff = df_ft_numeric - df_non_ft_numeric

            # Add a column to identify the pair
            df_diff['base_name'] = base_name
            df_diff['layer_id'] = df_ft_numeric['layer_id']

            # Optional: Reset index to ensure a proper stacking
            df_diff.reset_index(drop=True, inplace=True)

            # Append to the list
            list_differences.append(df_diff)

    # Concatenate all the difference DataFrames
    if list_differences:
        df_all_differences = pd.concat(list_differences, ignore_index=True)
    else:
        df_all_differences = pd.DataFrame()  # Return empty DataFrame if no differences found

    return df_all_differences


In [ ]:
la_cf10 = delta_merge_layers(df_path_cf10, 'fl')

In [ ]:
la_cf10 = la_cf10[['layer_id', 'alpha',  'entropy', 'log_norm', 'log_spectral_norm', 'base_name']]

In [ ]:
la_d_cf10 = la_cf10.drop(la_cf10[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']][(la_cf10[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']] == 0).all(axis=1)].index)

In [ ]:
def compute_pairwise_differences(df):
    """
    Computes pairwise differences between all categories and 'bs' based on 'base_name',
    for matching 'layer_id's.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing 'base_name', 'layer_id', and numeric columns.

    Returns:
        pd.DataFrame: A DataFrame containing the differences with columns 'base_name', 'layer_id', and difference columns.
    """
    # Identify numeric columns excluding 'layer_id'
    numeric_cols = df.select_dtypes(include='number').columns.tolist()
    if 'layer_id' in numeric_cols:
        numeric_cols.remove('layer_id')

    # Separate 'bs' and other categories
    df_bs = df[df['base_name'] == 'bs']
    df_others = df[df['base_name'] != 'bs']

    # Merge on 'layer_id'
    df_merged = pd.merge(
        df_others,
        df_bs,
        on='layer_id',
        suffixes=('', '_bs'),
        how='inner'  # Ensure only matching 'layer_id's are joined
    )

    # Compute differences for numeric columns
    for col in numeric_cols:
        df_merged[f'{col}_diff'] = df_merged[col] - df_merged[f'{col}_bs']

    # Prepare the final DataFrame
    cols_to_keep = ['base_name', 'layer_id'] + [f'{col}_diff' for col in numeric_cols]
    df_differences = df_merged[cols_to_keep]

    # Reset index and sort (optional)
    df_differences.reset_index(drop=True, inplace=True)
    df_differences.sort_values(by=['base_name', 'layer_id'], inplace=True)

    return df_differences

In [ ]:
la_dif_cf10 = compute_pairwise_differences(la_d_cf10)

In [ ]:
def categorize_index(index_value):
    if 'div' in index_value:
        return 'div'
    elif 'dp' in index_value:
        return 'dp'
    elif 'wd' in index_value:
        return 'wd'
    else:
        return 'bs'  # Optional: in case none of the patterns match

# Apply the function to the index and create a new column
la_dif_cf10['category'] = la_dif_cf10['base_name'].map(categorize_index)

In [ ]:
la_avg_cf10 = la_dif_cf10.drop(['base_name'], axis=1).groupby(['layer_id', 'category'], as_index=False).mean()
la_avg_cf10

In [ ]:
la_avg_cf10.loc[la_avg_cf10['layer_id'] == 343, 'layer_id'] = 235

In [ ]:
group_means = la_avg_cf10.iloc[:, 1:].groupby('category').mean()
data = group_means.to_dict(orient='list')
data 

In [ ]:
# Create the DataFrame
data = {'alpha_diff': [0.19615178728245358,
  0.20241053582139076,
  -0.02809366705473737],
 'entropy_diff': [2.327323534914605e-05,
  1.115623929031917e-05,
  1.3542505331875449e-05],
 'log_norm_diff': [-4.453021505305202e-05,
  -3.4735553717987e-05,
  -0.005059927762346135],
 'log_spectral_norm_diff': [-0.0009528485750048703,
  -0.0006574288014189499,
  -0.005862142038832641]}

df = pd.DataFrame(data, index=['div', 'dp', 'wd'])

# Set up subplots: 1 row, 5 columns
fig, axes = plt.subplots(1, 4, figsize=(18, 4), sharey=False)

# Plot each metric separately
for i, col in enumerate(df.columns):
    ax = axes[i]
    
    # Assign colors: dark blue if negative, skyblue otherwise
    colors = ['darkblue' if val < 0 else 'skyblue' for val in df[col]]
    
    df[col].plot(kind='bar', ax=ax, color=colors, edgecolor='black')
    
    ax.set_title(col, fontsize=12)
    ax.set_xticks(range(len(df.index)))
    ax.set_xticklabels(df.index, rotation=45, fontsize=10)
    ax.tick_params(axis='y', labelsize=9)
    ax.grid(axis='y', linestyle='--', alpha=0.5)


plt.tight_layout()  

plt.show()


In [ ]:
la_avg_cf10[la_avg_cf10['layer_id']==235].iloc[:, 2:].to_dict(orient='list')

In [ ]:
# Create the DataFrame
data = {'alpha_diff': [11.369901124842798, 11.36530840197412, 0.0071598491011031955],
 'entropy_diff': [0.00012308618644237823,
  0.00033468425816210456,
  0.00020204603872590413],
 'log_norm_diff': [-0.005957868924372866,
  -0.002637470294675426,
  -0.0007767100580427141],
 'log_spectral_norm_diff': [-0.007025681791097915,
  -0.011477741864477684,
  -0.006846545316608212]}

df = pd.DataFrame(data, index=['div', 'dp', 'wd'])

# Set up subplots: 1 row, 5 columns
fig, axes = plt.subplots(1, 4, figsize=(18, 4), sharey=False)

# Plot each metric separately
for i, col in enumerate(df.columns):
    ax = axes[i]
    
    # Assign colors: dark blue if negative, skyblue otherwise
    colors = ['darkblue' if val < 0 else 'skyblue' for val in df[col]]
    
    df[col].plot(kind='bar', ax=ax, color=colors, edgecolor='black')
    
    ax.set_title(col, fontsize=12)
    ax.set_xticks(range(len(df.index)))
    ax.set_xticklabels(df.index, rotation=45, fontsize=10)
    ax.tick_params(axis='y', labelsize=9)
    ax.grid(axis='y', linestyle='--', alpha=0.5)


plt.tight_layout()  

plt.show()

cf100

In [ ]:
df_path_cf100 = 'cf100-rn/ww-df'
la_cf100 = delta_merge_layers(df_path_cf100, 'fl')

In [ ]:
la_cf100 = la_cf100[['layer_id', 'alpha',  'entropy', 'log_norm', 'log_spectral_norm', 'base_name']]
la_cf100

In [ ]:
la_d_cf100 = la_cf100.drop(la_cf100[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']][(la_cf100[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']] == 0).all(axis=1)].index)

In [ ]:
la_dif_cf100 = compute_pairwise_differences(la_d_cf100)
la_dif_cf100['category'] = la_dif_cf100['base_name'].map(categorize_index)

In [ ]:
la_avg_cf100 = la_dif_cf100.drop(['base_name'], axis=1).groupby(['layer_id', 'category'], as_index=False).mean()
la_avg_cf100

In [ ]:
la_avg_cf100.loc[la_avg_cf100['layer_id'] == 343, 'layer_id'] = 235

In [ ]:
group_means = la_avg_cf100.iloc[:, 1:].groupby('category').mean()
data = group_means.to_dict(orient='list')
data 

In [ ]:
data = {'alpha_diff': [0.04007498732014961,
  -0.025550324583770972,
  -0.02823550418773014],
 'entropy_diff': [1.598354475784103e-05,
  6.141429749904429e-06,
  -4.942509113818755e-06],
 'log_norm_diff': [-7.990164818008211e-05,
  -4.915728179148098e-05,
  -0.007226685293042139],
 'log_spectral_norm_diff': [-0.0009966920705268341,
  -0.000575728033980564,
  -0.007843973865608006]}

df = pd.DataFrame(data, index=['div', 'dp', 'wd'])

# Set up subplots: 1 row, 5 columns
fig, axes = plt.subplots(1, 4, figsize=(18, 4), sharey=False)

# Plot each metric separately
for i, col in enumerate(df.columns):
    ax = axes[i]
    
    # Assign colors: dark blue if negative, skyblue otherwise
    colors = ['darkblue' if val < 0 else 'skyblue' for val in df[col]]
    
    df[col].plot(kind='bar', ax=ax, color=colors, edgecolor='black')
    
    ax.set_title(col, fontsize=12)
    ax.set_xticks(range(len(df.index)))
    ax.set_xticklabels(df.index, rotation=45, fontsize=10)
    ax.tick_params(axis='y', labelsize=9)
    ax.grid(axis='y', linestyle='--', alpha=0.5)


plt.tight_layout()  


plt.show()

In [ ]:
la_avg_cf100[la_avg_cf100['layer_id']==235].iloc[:, 2:].to_dict(orient='list')

In [ ]:
data = {'alpha_diff': [5.446829494328063, 2.7595297895703803, 5.167907631804872],
 'entropy_diff': [3.314444996035393e-05,
  2.360637211906713e-05,
  -4.1050536885171386e-05],
 'log_norm_diff': [-0.003382980269840541,
  -0.001746870591053773,
  -0.004225974899703333],
 'log_spectral_norm_diff': [0.0035734891010180845,
  0.0007648543106333616,
  0.0023209949530842433]}

df = pd.DataFrame(data, index=['div', 'dp', 'wd'])

# Set up subplots: 1 row, 5 columns
fig, axes = plt.subplots(1, 4, figsize=(18, 4), sharey=False)

# Plot each metric separately
for i, col in enumerate(df.columns):
    ax = axes[i]
    
    # Assign colors: dark blue if negative, skyblue otherwise
    colors = ['darkblue' if val < 0 else 'skyblue' for val in df[col]]
    
    df[col].plot(kind='bar', ax=ax, color=colors, edgecolor='black')
    
    ax.set_title(col, fontsize=12)
    ax.set_xticks(range(len(df.index)))
    ax.set_xticklabels(df.index, rotation=45, fontsize=10)
    ax.tick_params(axis='y', labelsize=9)
    ax.grid(axis='y', linestyle='--', alpha=0.5)


plt.tight_layout()  


plt.show()

cars

In [ ]:
df_path_car = 'car_train/ww-df'
la_car = delta_merge_layers(df_path_car, 'int')

la_car = la_car[['layer_id', 'alpha',  'entropy', 'log_norm', 'log_spectral_norm', 'base_name']]
la_car

In [ ]:
la_d_car = la_car.drop(la_car[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']][(la_car[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']] == 0).all(axis=1)].index)

la_dif_car = compute_pairwise_differences(la_d_car)
la_dif_car['category'] = la_dif_car['base_name'].map(categorize_index)

la_dif_car


In [ ]:
la_avg_car = la_dif_car.drop(['base_name'], axis=1).groupby(['layer_id', 'category'], as_index=False).mean()
la_avg_car

In [ ]:
la_avg_car.loc[la_avg_car['layer_id'] == 343, 'layer_id'] = 235

In [ ]:
group_means = la_avg_car.iloc[:, 1:].groupby('category').mean()
data = group_means.to_dict(orient='list')
data 

In [ ]:
data = {'alpha_diff': [-0.015176526606014281,
  0.09818165875198252,
  0.06014596658546485],
 'entropy_diff': [-1.0457865369182613e-06,
  -2.991842930343917e-06,
  -1.9225579384149174e-05],
 'log_norm_diff': [9.058754011337699e-05,
  7.849807852915328e-06,
  -0.01037723289092589],
 'log_spectral_norm_diff': [-1.3266491693967473e-05,
  -0.00021858936378907993,
  -0.011334037820567133]}

df = pd.DataFrame(data, index=['div', 'dp', 'wd'])

# Set up subplots: 1 row, 5 columns
fig, axes = plt.subplots(1, 4, figsize=(18, 4), sharey=False)

# Plot each metric separately
for i, col in enumerate(df.columns):
    ax = axes[i]
    
    # Assign colors: dark blue if negative, skyblue otherwise
    colors = ['darkblue' if val < 0 else 'skyblue' for val in df[col]]
    
    df[col].plot(kind='bar', ax=ax, color=colors, edgecolor='black')
    
    ax.set_title(col, fontsize=12)
    ax.set_xticks(range(len(df.index)))
    ax.set_xticklabels(df.index, rotation=45, fontsize=10)
    ax.tick_params(axis='y', labelsize=9)
    ax.grid(axis='y', linestyle='--', alpha=0.5)

plt.tight_layout()  

plt.show()

In [ ]:
la_avg_car[la_avg_car['layer_id']==235].iloc[:, 2:].to_dict(orient='list')

In [ ]:
data = {'alpha_diff': [0.16803314364034777, 3.288227879802239, 2.4036791996943787],
 'entropy_diff': [2.1726904837526816e-05,
  -3.0047501142743682e-05,
  -4.6939103387488856e-07],
 'log_norm_diff': [-0.0010641326592188305,
  -0.001969499599642932,
  -0.006753559363009291],
 'log_spectral_norm_diff': [-0.004071674126778063,
  -0.0034588300969596517,
  -0.009569524947168367]}

df = pd.DataFrame(data, index=['div', 'dp', 'wd'])

# Set up subplots: 1 row, 5 columns
fig, axes = plt.subplots(1, 4, figsize=(18, 4), sharey=False)

# Plot each metric separately
for i, col in enumerate(df.columns):
    ax = axes[i]
    
    # Assign colors: dark blue if negative, skyblue otherwise
    colors = ['darkblue' if val < 0 else 'skyblue' for val in df[col]]
    
    df[col].plot(kind='bar', ax=ax, color=colors, edgecolor='black')
    
    ax.set_title(col, fontsize=12)
    ax.set_xticks(range(len(df.index)))
    ax.set_xticklabels(df.index, rotation=45, fontsize=10)
    ax.tick_params(axis='y', labelsize=9)
    ax.grid(axis='y', linestyle='--', alpha=0.5)


plt.tight_layout()  


plt.show()

Dom

In [ ]:
def delta_merge_layers_(folder_path):
    # Initialize dictionaries to store DataFrames for ft and non-ft files
    dataframes_ft = {}
    dataframes_non_ft = {}

    # List all CSV files in the folder
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Loop through each file and categorize it as 'ft' or 'non-ft'
    for file in files:
        # Check if file matches the ft pattern
        if file.startswith('ft_'):  # Identify "ft" files
            base_name = file.replace('ft_', '').replace('.csv', '')  # Extract base name
            file_path = os.path.join(folder_path, file)
            # Read the entire DataFrame from the ft file
            df = pd.read_csv(file_path)
            if not df.empty:
                dataframes_ft[base_name] = df
        # Check if file matches the non-ft pattern
        else:  # Identify "non-ft" files
            base_name = file.replace('.csv', '')  # Extract base name
            file_path = os.path.join(folder_path, file)
            # Read the entire DataFrame from the non-ft file
            df = pd.read_csv(file_path)
            if not df.empty:
                dataframes_non_ft[base_name] = df

    # Initialize a list to store the differences DataFrames
    list_differences = []

    # Compute the differences for each matching pair
    for base_name in dataframes_ft:
        if base_name in dataframes_non_ft:
            df_ft = dataframes_ft[base_name]
            df_non_ft = dataframes_non_ft[base_name]

            # Align DataFrames on both axes (rows and columns)
            df_ft_aligned, df_non_ft_aligned = df_ft.align(df_non_ft, join='outer', axis=None, fill_value=0)

            # Convert boolean columns to integers
            for df in [df_ft_aligned, df_non_ft_aligned]:
                bool_cols = df.select_dtypes(include=['bool']).columns
                df[bool_cols] = df[bool_cols].astype(int)

            # Convert all columns to numeric, coercing errors to NaN
            df_ft_numeric = df_ft_aligned.apply(pd.to_numeric, errors='coerce')
            df_non_ft_numeric = df_non_ft_aligned.apply(pd.to_numeric, errors='coerce')

            # Fill NaN values with 0 (optional, depending on how you want to handle missing values)
            df_ft_numeric = df_ft_numeric.fillna(0)
            df_non_ft_numeric = df_non_ft_numeric.fillna(0)

            # Compute the difference between the ft and non-ft DataFrames
            df_diff = df_ft_numeric - df_non_ft_numeric

            # Add a column to identify the pair
            df_diff['base_name'] = base_name
            df_diff['layer_id'] = df_ft_numeric['layer_id']

            # Optional: Reset index to ensure a proper stacking
            df_diff.reset_index(drop=True, inplace=True)

            # Append to the list
            list_differences.append(df_diff)

    # Concatenate all the difference DataFrames
    if list_differences:
        df_all_differences = pd.concat(list_differences, ignore_index=True)
    else:
        df_all_differences = pd.DataFrame()  # Return empty DataFrame if no differences found

    return df_all_differences


In [ ]:
df_path_dom = 'domainnet/ww-df'
la_dom = delta_merge_layers_(df_path_dom)

la_dom = la_dom[['layer_id', 'alpha',  'entropy', 'log_norm', 'log_spectral_norm', 'base_name']]
la_dom

In [ ]:
la_d_dom = la_dom.drop(la_dom[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']][(la_dom[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']] == 0).all(axis=1)].index)

la_dif_dom = compute_pairwise_differences(la_d_dom)
la_dif_dom['category'] = la_dif_dom['base_name'].map(categorize_index)


la_avg_dom = la_dif_dom.drop(['base_name'], axis=1).groupby(['layer_id', 'category'], as_index=False).mean()
la_avg_dom

In [ ]:
la_avg_dom.loc[la_avg_dom['layer_id'] == 343, 'layer_id'] = 235

In [ ]:
group_means = la_avg_dom.iloc[:, 1:].groupby('category').mean()
data = group_means.to_dict(orient='list')
data 

In [ ]:
data = {'alpha_diff': [0.06794499960398517, 0.04991637557500118, 0.0755418279415309],
 'entropy_diff': [2.2012830582528823e-05,
  3.823611175338924e-05,
  2.2647003354770497e-05],
 'log_norm_diff': [1.0686380514502356e-05,
  -5.656753066055452e-05,
  -0.00935656892948474],
 'log_spectral_norm_diff': [-3.662837371960647e-05,
  -0.00023505300497649525,
  -0.010364544913115281]}

df = pd.DataFrame(data, index=['div', 'dp', 'wd'])

# Set up subplots: 1 row, 5 columns
fig, axes = plt.subplots(1, 4, figsize=(18, 4), sharey=False)

# Plot each metric separately
for i, col in enumerate(df.columns):
    ax = axes[i]
    
    # Assign colors: dark blue if negative, skyblue otherwise
    colors = ['darkblue' if val < 0 else 'skyblue' for val in df[col]]
    
    df[col].plot(kind='bar', ax=ax, color=colors, edgecolor='black')
    
    ax.set_title(col, fontsize=12)
    ax.set_xticks(range(len(df.index)))
    ax.set_xticklabels(df.index, rotation=45, fontsize=10)
    ax.tick_params(axis='y', labelsize=9)
    ax.grid(axis='y', linestyle='--', alpha=0.5)


plt.tight_layout()  

plt.show()

In [ ]:
la_avg_dom[la_avg_dom['layer_id']==235].iloc[:, 2:].to_dict(orient='list')

In [ ]:
data = {'alpha_diff': [-0.3786984277649423, 1.0976138293294104, 1.4947644932433797],
 'entropy_diff': [-6.841821662910697e-05,
  -5.711743336703212e-05,
  -9.957549481418952e-05],
 'log_norm_diff': [-0.0001338371351972345,
  -0.0030829175168491374,
  -0.002882332304170713],
 'log_spectral_norm_diff': [0.004298671625529077,
  -0.00021436323770364363,
  -0.0027375511883758106]}

df = pd.DataFrame(data, index=['div', 'dp', 'wd'])

# Set up subplots: 1 row, 5 columns
fig, axes = plt.subplots(1, 4, figsize=(18, 4), sharey=False)

# Plot each metric separately
for i, col in enumerate(df.columns):
    ax = axes[i]
    
    # Assign colors: dark blue if negative, skyblue otherwise
    colors = ['darkblue' if val < 0 else 'skyblue' for val in df[col]]
    
    df[col].plot(kind='bar', ax=ax, color=colors, edgecolor='black')
    
    ax.set_title(col, fontsize=12)
    ax.set_xticks(range(len(df.index)))
    ax.set_xticklabels(df.index, rotation=45, fontsize=10)
    ax.tick_params(axis='y', labelsize=9)
    ax.grid(axis='y', linestyle='--', alpha=0.5)


plt.tight_layout()  

plt.show()

In [ ]:
yaxis = ['alpha_diff', 'entropy_diff', 'log_norm_diff',
         'log_spectral_norm_diff']

var = ['Alpha', 'Entropy', 'Log Frobenius norm',
         'Log Spectral Norm']

for idx, y in enumerate(yaxis): 
    sns.lineplot(
        data=la_avg_car,
        x='layer_id',
        y=y, 
        hue='category',
        style='category',
        markers=True,
        dashes=False,   
        palette='Set2'
    )

    plt.xlabel('Layer ID', fontsize=14)
    plt.ylabel(f'{var[idx]}', fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    
    # Let matplotlib choose best position for the legend inside the figure
    plt.legend(title='Category', loc='best', fontsize=12, title_fontsize=12)

    plt.grid(True)
    plt.tight_layout()
     # Save with high resolution
    plt.show()

In [ ]:
for idx, y in enumerate(yaxis): 
    sns.lineplot(
        data=la_avg_cf100,
        x='layer_id',
        y=y, 
        hue='category',
        style='category',
        markers=True,
        dashes=False,   
        palette='Set2'
    )

    plt.xlabel('Layer ID', fontsize=14)
    plt.ylabel(f'{var[idx]}', fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    
    # Let matplotlib choose best position for the legend inside the figure
    plt.legend(title='Category', loc='best', fontsize=12, title_fontsize=12)

    plt.grid(True)
    plt.tight_layout()


    plt.show()

In [ ]:
for idx, y in enumerate(yaxis): 
    sns.lineplot(
        data=la_avg_cf10,
        x='layer_id',
        y=y, 
        hue='category',
        style='category',
        markers=True,
        dashes=False,   
        palette='Set2'
    )

    plt.xlabel('Layer ID', fontsize=14)
    plt.ylabel(f'{var[idx]}', fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    
    # Let matplotlib choose best position for the legend inside the figure
    plt.legend(title='Category', loc='best', fontsize=12, title_fontsize=12)

    plt.grid(True)
    plt.tight_layout()

     # Save with high resolution
    plt.show()

In [ ]:
yaxis = ['alpha_diff', 'entropy_diff', 'log_norm_diff',
         'log_spectral_norm_diff']

var = ['Alpha', 'Entropy', 'Log Frobenius norm',
         'Log Spectral Norm']

for idx, y in enumerate(yaxis): 
    sns.lineplot(
        data=la_avg_dom,
        x='layer_id',
        y=y, 
        hue='category',
        style='category',
        markers=True,
        dashes=False,   
        palette='Set2'
    )

    plt.xlabel('Layer ID', fontsize=14)
    plt.ylabel(f'{var[idx]}', fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    
    # Let matplotlib choose best position for the legend inside the figure
    plt.legend(title='Category', loc='best', fontsize=12, title_fontsize=12)

    plt.grid(True)
    plt.tight_layout()

     # Save with high resolution
    plt.show()
